# Goal:
This notebook further filter the data by criteria:
1. Calculate the image sizes. smaller than 2000 b is not considered (too few information)
2. Assign the road class to each panoid. Remove the highway only ones
3. Use the segmentation results to find the ones that with at least one building. Label the ones without any building as outlier

In [1]:
import tqdm
import geopandas as gpd
import os
import tqdm
import numpy as np
import pandas as pd
import sys
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
from utils.gsvload import GSVSummary
from shapely.geometry import Point
from multiprocessing import Pool
import concurrent.futures

RAW_FOLDER_ROOT = "/lustre1/g/geog_pyloo/05_timemachine/GSV/gsv_rgb"
ROAD_PTH = "{RAW_FOLDER_ROOT}/{cityabbrlower}/road/osm.geojson"
# TGT_FILE = "gsv_pano_label.csv" 
TGT_FILE = "{cityabbrlower}_meta.csv"

In [4]:
# city = "Milan"
# roaddf = gpd.read_file(ROAD_PTH.format(
#     RAW_FOLDER_ROOT = RAW_FOLDER_ROOT,
#     cityabbrlower = city.lower().replace(" ", "")
# ))
# roaddf

ERROR 1: PROJ: proj_create_from_database: Open of /lustre1/u/yuanzf/anaconda3/envs/geo311/share/proj failed


,u,v,osmid,name,length,length_meter,newID,geometry
0,10371529,743371634,274433644,Via Novara,154.435,220.533621,274433644_0,"LINESTRING (1010405.229 5698607.719, 1010189.6..."
1,10371529,743371622,26703159,Via Novara,150.466,214.866824,26703159_1,"LINESTRING (1010405.229 5698607.719, 1010347.7..."
2,10371530,2754112526,270390223,Via Novara,29.791,42.535290,270390223_2,"LINESTRING (1013236.094 5697377.256, 1013196.4..."
3,13595397,10694405410,195654893,Viale Certosa,10.544,15.059787,195654893_3,"LINESTRING (1016420.500 5700507.562, 1016433.1..."
4,13595397,271096577,195654896,Viale Certosa,41.593,59.408124,195654896_4,"LINESTRING (1016420.500 5700507.562, 1016370.6..."
...,...,...,...,...,...,...,...,...
259962,11748228776,2162724465,1138284089_206271098_1138284090,nan,7.729,11.042054,1138284089_206271098_1138284090_259962,"LINESTRING (1022453.671 5702740.790, 1022454.4..."
259963,11748228776,2162688055,206271098_1138284102_1138284103,nan,13.096,18.709031,206271098_1138284102_1138284103_259963,"LINESTRING (1022453.671 5702740.790, 1022453.0..."
259964,11748228777,11748228776,135032696,Via Ettore Majorana,3.585,5.121567,135032696_259964,"LINESTRING (1022458.502 5702739.090, 1022453.6..."
259965,11748228777,2951516015,1016889293,nan,7.755,11.079938,1016889293_259965,"LINESTRING (1022458.502 5702739.090, 1022462.3..."


In [2]:
serviceaccount = "../../google_drive_personal.json"
import gspread

# from oauth2client.service_account import ServiceAccountCredentials
gc = gspread.service_account(filename=serviceaccount)


def read_url(url, SHEET_NAME):
    SHEET_ID = url.split("/")[5]
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    rows = worksheet.get_all_records()
    df_spread = pd.DataFrame(rows)
    return df_spread, worksheet


url = "https://docs.google.com/spreadsheets/d/1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw/edit?usp=sharing"
SHEETNAME = "select_city"
city_meta, other_worksheet = read_url(url, SHEETNAME)
city_meta = city_meta[city_meta['City']!=''].reset_index(drop = True)

## 1. Check image sizes

In [21]:
def get_gsv_file_size(city):
    citylower = city.lower().replace(" ", "")
    meta_path = "{RAW_FOLDER_ROOT}/{cityabbrlower}/gsvmeta/{tgt_tile}".format(
        RAW_FOLDER_ROOT = RAW_FOLDER_ROOT,
        cityabbrlower = citylower,
        tgt_tile = TGT_FILE.format(cityabbrlower = citylower)
    )

    metadf = pd.read_csv(meta_path)
    
    def get_file_size(file):
        try:
            return file, os.path.getsize(file)
        except OSError as e:
            print(f"Error: {e}")
            return file, None

    if './data/' in metadf["path"].values[0]:
        metadf['path'] = metadf['path'].apply(lambda x: x.replace("./data/", "/lustre1/g/geog_pyloo/05_timemachine/GSV/"))
        print("path fixed")
    files = metadf["path"].values
    if "size" in metadf.columns:
        print("size finished, no need to calculate. continue next")
        return None
    else:
        
        # Store file sizes in a dictionary
        file_sizes = {}
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # The map method helps maintain the order of results
            results = executor.map(get_file_size, files)

            for file, size in results:
                if size is not None:
                    file_sizes[file] = size
                else:
                    file_sizes[file] = 0  # the file is removed

        metadf["size"] = metadf["path"].apply(lambda x: file_sizes[x])
        metadf_update.to_csv(meta_path, index = False)
        return metadf

In [4]:
cityls = city_meta[city_meta['GSV Downloaded']>0]['City'].unique()
len(cityls)

127

In [ ]:
city = "Accra"

In [20]:
list(cityls)[65]

'Nagoya'

In [ ]:
from tqdm import tqdm
for city in tqdm(cityls[2:]):
    get_gsv_file_size(city)

  4%|▍         | 5/125 [1:25:10<32:59:08, 989.57s/it] 

size finished, no need to calculate. continue next


  6%|▋         | 8/125 [1:41:54<15:28:49, 476.32s/it]

size finished, no need to calculate. continue next


 28%|██▊       | 35/125 [6:12:46<14:41:25, 587.61s/it]

In [ ]:
city